In [2]:
import os
import numpy as np
import tensorflow as tf
from keras.models import Sequential
import pathlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
#import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, BinaryAccuracy, Precision, Recall, AUC
from sklearn.preprocessing import MinMaxScaler

sns.set_context("talk")

# MLFPM symposium
### Student workshop - Part 1

In this first part, we'll focus on classifying medical images using a few different approaches. We'll use a kaggle dataset containing [chest X-ray images](https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database) from COVID and viral pneumonia patients, and set to classify between the two using a baseline (Principal Component Analysis + Logistic Regression) and a state of the art model (a convolutional neural network called [ChexNet](https://arxiv.org/pdf/1711.05225.pdf)).
After comparing and discussing differences in performance, we'll also explore some explainability methods to test how well we can understand how our models make decisions.

Let's start!

### Set up the environment

In [ ]:
!git clone https://github.com/mlfpm/MLFPM_symposium_workshop

In [3]:
repo_path = "MLFPM_symposium_workshop/Part 1 - Chest Xray classification and explainability/"

### Download data from Kaggle

Run the cells below, and when prompted, use these Kaggle authentication details:

username: mlfpmitn

key: 3f880a8750f53ce7a6251356d17098e0

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as op
op.download("https://www.kaggle.com/tawsifurrahman/covid19-radiography-database")

### Preprocess images

In [ ]:
dataset_folder = os.path.join("covid19-radiography-database/COVID-19_Radiography_Dataset")

In [ ]:
files_not_important = ["COVID.metadata.xlsx",
                       "Lung_Opacity.metadata.xlsx",
                       "Normal.metadata.xlsx",
                       "README.md.txt",
                       "Viral Pneumonia.metadata.xlsx"]
for i in files_not_important:
    os.remove(os.path.join(dataset_folder, i))

In [ ]:
import shutil
files_not_important = [
                       "COVID/masks",
                       "Lung_Opacity",
                       "Normal",
                       "Viral Pneumonia/masks"]
for i in files_not_important:
    shutil.rmtree(os.path.join(dataset_folder, i), ignore_errors=True)

In [ ]:
datasetObject = pathlib.Path(os.path.join(dataset_folder))
images = list(datasetObject.glob("*/*/*.*"))

In [ ]:
image_data_generator = ImageDataGenerator(
    rescale = 1/255, vertical_flip= False, horizontal_flip=True, zoom_range=0.1, zca_whitening=False,
    samplewise_center=True, samplewise_std_normalization=True, validation_split= 0.1,
    rotation_range=0.2)
training_dataset = image_data_generator.flow_from_directory(
    dataset_folder, target_size = (224, 224), color_mode ='rgb',subset='training', batch_size=8, shuffle=True
)
validation_dataset = image_data_generator.flow_from_directory(
    dataset_folder,  target_size=(224, 224), color_mode = 'rgb', subset='validation', batch_size = 8, shuffle = True
)

In [ ]:
label_dict = {
    (0, 1):"COVID",
    (1, 0):"Viral pneumonia"
}

In [ ]:
print('COVID cases:', (training_dataset.labels == 0).sum())
print('Viral pneumonia cases:', (training_dataset.labels == 1).sum())

In [ ]:
single_batch = training_dataset.next()
images = single_batch[0]
label = single_batch[1]
plt.figure(figsize = (20, 10))
for i in range(8):
    plt.subplot(2, 4, (i + 1))
    plt.imshow(np.clip(images[i], 0, 1))
    plt.title(label_dict[tuple(label[i])])
plt.show()

## Classification models

### Principal Component Analysis + Logistic Regression baseline

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
n_train_batches = 200
n_val_batches = 50
X_train = []
y_train = []

X_val = []
y_val = []

pca_training_dataset = image_data_generator.flow_from_directory(
    dataset_folder, target_size = (224, 224), color_mode ='grayscale',subset='training', batch_size=8, shuffle=True
)
pca_validation_dataset = image_data_generator.flow_from_directory(
    dataset_folder, target_size = (224, 224), color_mode ='grayscale',subset='validation', batch_size=8, shuffle=True
)


for _ in range(n_train_batches):
    images, labels = pca_training_dataset.next()
    X_train.extend(images.reshape(8,-1))
    y_train.extend(labels)
    
for _ in range(n_val_batches):  
    val_images, val_labels = pca_validation_dataset.next()
    X_val.extend(val_images.reshape(8,-1))
    y_val.extend(val_labels)
    
X_train = np.vstack(X_train)
X_train -= np.mean(X_train, axis=0)
y_train = np.vstack(y_train)
X_val = np.vstack(X_val)
X_val -= np.mean(X_val, axis=0)
y_val = np.vstack(y_val)

In [ ]:
pca = PCA(n_components=100)
pca.fit(X_train)

fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(range(30), np.cumsum(pca.explained_variance_ratio_[:30]))
ax.set_ylim(0,1)
ax.set_ylabel("Cumulative Explained Variance Ratio")
ax.set_xlabel("N. Components")
ax.set_xticks(list(range(0,20,2)))
plt.show()


In [ ]:
n_components = 32
eigenimages = pca.components_[:n_components]

# Show the first 9 eigenvectors
fig, axes = plt.subplots(3,3,sharex=True,sharey=True,figsize=(16,20))
for i in range(9):
    axes[i//3][i%3].imshow(eigenimages[i].reshape(224,224), cmap="gray")
    axes[i//3][i%3].set_title("Eig. {}".format(i+1))
    
fig.suptitle("First 9 eigenvectors (images) of the PCA projection", y=0.9, fontsize=30)
plt.show()

In [ ]:
pca_embeddings_train = pca.transform(X_train)
pca_embeddings_val = pca.transform(X_val)
fig, axs = plt.subplots(1, 2, figsize=(14,7))

ix = np.where(y_train[:,0]==1)
neg_ix = np.where(y_train[:,0]==0)
axs[0].scatter(pca_embeddings_train[ix, 0], pca_embeddings_train[ix,1], s=80, color="#ff7f0e", label="Positive")
axs[0].scatter(pca_embeddings_train[neg_ix, 0], pca_embeddings_train[neg_ix,1], s=80, color="#1f77b4", label="Negative")
axs[0].set_title("First 2 PCs of training data")

ix = np.where(y_val[:,0]==1)
neg_ix = np.where(y_val[:,0]==0)
axs[1].scatter(pca_embeddings_train[ix, 0], pca_embeddings_train[ix,1], s=80, c="#ff7f0e")
axs[1].scatter(pca_embeddings_train[neg_ix, 0], pca_embeddings_train[neg_ix,1], s=80, color="#1f77b4")
axs[1].set_title("First 2 PCs of validation data")
fig.legend(loc="lower right")

plt.tight_layout()
plt.show()


In [ ]:
print("Training Logistic Regression model")
lr = LogisticRegression(max_iter=2500)
lr.fit(pca_embeddings_train, y_train[:,0])

predictions = lr.predict(pca_embeddings_val)

print("Validation Accuracy: {:.2%}".format(accuracy_score(y_val[:,0], predictions)))


### Build and train a CNN classification model

We'll use a CNN, give some details bla bla

In [ ]:
from keras.applications import densenet
from keras.initializers import GlorotNormal
d = densenet.DenseNet121(weights=None, include_top = False, input_shape = (224, 224, 3))

In [ ]:
print(d.output_shape)
m = tf.keras.layers.Dropout(0.7)(d.output)
m = tf.keras.layers.GlobalAveragePooling2D()(m)                         
m = tf.keras.layers.Dropout(0.7)(m)
m = tf.keras.layers.Dense(2, kernel_initializer=GlorotNormal(),
                          activation = 'softmax', kernel_regularizer= tf.keras.regularizers.L2(0.0001),
                          bias_regularizer= tf.keras.regularizers.L2(0.0001))(m)
m = tf.keras.models.Model(inputs = d.input, outputs = m)

m.load_weights(os.path.join(repo_path, "CheXNet_Keras_0.3.0_raw_weights.h5"), by_name=True, skip_mismatch=True)

for layer in m.layers[:200]:
    layer.trainable = False
for layer in m.layers[200:]:
    layer.trainable = True

In [ ]:
m.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), 
    loss = 'categorical_crossentropy', 
    metrics =  [
        TruePositives(name='tp'),
        FalsePositives(name='fp'),
        TrueNegatives(name='tn'),
        FalseNegatives(name='fn'), 
        'accuracy',
        Precision(name='precision'),
        Recall(name='recall')
      ]
    )

In [ ]:
ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1, 
    mode = 'min',
    patience= 2
)

In [ ]:
history = m.fit(
    training_dataset,
    validation_data = validation_dataset,
    batch_size = 8,
    epochs = 1, # Set to ~25 to reach good performance. Takes around 30 min on a GPU environment
    callbacks = [ReduceLROnPlateau, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, mode = 'min', restore_best_weights=True)]
)

In [ ]:
# plt.figure(figsize = (10, 5))
# plt.plot(history.history['accuracy'], label="accuracy")
# plt.plot(history.history['val_accuracy'], label="val_accuracy")

# plt.xlabel("epochs")

# plt.legend()

In [ ]:
# plt.figure(figsize = (10, 5))
# plt.plot(history.history['loss'], label = "loss")
# plt.plot(history.history['val_loss'], label = "val_loss")

# plt.xlabel("epochs")

# plt.legend()

In [ ]:
# Load fine tuned weights
m.load_weights(os.path.join(repo_path, "CheXNet_Keras_0.3.0_tuned_weights.h5"), by_name=True, skip_mismatch=True)

In [ ]:
m.evaluate(validation_dataset, batch_size = 8)

### Performance comparison

### Model interpretation

##### 1. Manual error analysis

In [ ]:
# TBD (meeting with a doctor on Monday, to make it a bit more interesting)

##### 2. Saliency maps

In [ ]:
def get_saliency_map(model, image, class_idx):

    with tf.GradientTape() as tape:
        tape.watch(image)
        predictions = model(image)
        
        loss = predictions[:, class_idx]
    
    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, image)
    
    # take maximum across channels
    gradient = tf.reduce_max(gradient, axis=-1)
    
    # convert to numpy
    gradient = gradient.numpy()
    
    # normaliz between 0 and 1
    min_val, max_val = np.min(gradient), np.max(gradient)
    smap = (gradient - min_val) / (max_val - min_val + tf.keras.backend.epsilon())
    
    return np.squeeze(smap)

In [ ]:
test_image = validation_dataset.next()
test_image, test_label = test_image[0][0], int(test_image[1][0][1])

saliency = get_saliency_map(m, tf.expand_dims(test_image, axis=0), test_label)

In [ ]:
# Visualize image and raw heatmaps

fig, (ax1, ax2,) = plt.subplots(1, 2, figsize=(16, 8))
sns.set_context("talk")

ax1.set_title("Current test image")
ax1.imshow(np.clip(test_image, 0, 1))

ax2.set_title("Obtained saliency maps")
ax2.imshow(saliency)

plt.show()

##### 3. Grad CAM

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):

    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
def gradcam_overlay(img, heatmap, alpha=0.4):

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    return superimposed_img

In [ ]:
test_image = validation_dataset.next()[0][0]

gradcam_outputs = make_gradcam_heatmap(
    tf.expand_dims(test_image, axis=0), 
    m, 
    "conv5_block16_2_conv", 
    pred_index=None
    )

gradcam_superimposed = gradcam_overlay(test_image, gradcam_outputs, alpha=0.01)

In [ ]:
# Visualize image and raw heatmaps

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 16))
sns.set_context("talk")

ax1.set_title("Current test image")
ax1.imshow(np.clip(test_image, 0, 1))

ax2.set_title("Raw Grad CAM heatmap")
ax2.imshow(gradcam_outputs)

ax3.set_title("Grad CAM superimposed")
ax3.imshow(gradcam_superimposed)

plt.show()